In [7]:
import fastf1
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
import warnings

# Suppress specific warnings from FastF1 and others if needed
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

# Enable FastF1 caching
try:
    fastf1.Cache.enable_cache("f1_cache")
except Exception as e:
    print(f"Error enabling cache: {e}. Check cache directory permissions.")

def get_standings_before_round(year, target_round):
    """Calculate driver and constructor points standings *before* a specific round."""
    driver_standings = {}
    constructor_standings = {}
    max_driver_points = 0
    max_constructor_points = 0
    try:
        schedule = fastf1.get_event_schedule(year, include_testing=False)
        races_before = schedule[schedule['RoundNumber'] < target_round]

        if races_before.empty:
            return {}, 0, {}, 0 # No races before this one

        for index, race in races_before.iterrows():
            try:
                session = fastf1.get_session(year, race['EventName'], 'R')
                session.load(laps=False, telemetry=False, weather=False, messages=False)
                results = session.results
                if results is not None and not results.empty:
                    for _, driver_result in results.iterrows():
                        driver = driver_result['Abbreviation']
                        team = driver_result['TeamName']
                        points = driver_result['Points']
                        # Update driver standings
                        driver_standings[driver] = driver_standings.get(driver, 0) + points
                        # Update constructor standings
                        constructor_standings[team] = constructor_standings.get(team, 0) + points
            except Exception as e:
                # Silently continue if a past race fails to load
                continue

        if driver_standings:
            max_driver_points = max(driver_standings.values()) if driver_standings else 0
        if constructor_standings:
             max_constructor_points = max(constructor_standings.values()) if constructor_standings else 0

    except Exception as e:
        print(f"Error calculating standings for {year} before round {target_round}: {e}")
        return {}, 0, {}, 0

    return driver_standings, max_driver_points, constructor_standings, max_constructor_points

def get_race_data(year, event_identifier):
    """Get Race and Qualifying data for a specific event."""
    try:
        session_r = fastf1.get_session(year, event_identifier, 'R')
        session_r.load(laps=False, telemetry=False, weather=False, messages=False)
        results_r = session_r.results

        if results_r is None or results_r.empty:
            print(f"No race results found for {year} {event_identifier}. Skipping.")
            return None

        results_r['FinishingPosition'] = pd.to_numeric(results_r['Position'], errors='coerce')
        results_r['GridPosition'] = pd.to_numeric(results_r['GridPosition'], errors='coerce')  # Add GridPosition
        results_r.dropna(subset=['FinishingPosition', 'GridPosition'], inplace=True)  # Ensure both positions are valid
        results_r[['FinishingPosition', 'GridPosition']] = results_r[['FinishingPosition', 'GridPosition']].astype(int)

        session_q = fastf1.get_session(year, event_identifier, 'Q')
        session_q.load(laps=False, telemetry=False, weather=False, messages=False)
        results_q = session_q.results

        if results_q is None or results_q.empty:
            print(f"No qualifying results found for {year} {event_identifier}. Proceeding without Quali times.")
            qualifying_times = pd.DataFrame({'Abbreviation': results_r['Abbreviation'], 'QualifyingTime (s)': np.nan})
        else:
            results_q['QualifyingTime'] = results_q[['Q1', 'Q2', 'Q3']].min(axis=1)
            results_q['QualifyingTime (s)'] = results_q['QualifyingTime'].dt.total_seconds()
            qualifying_times = results_q[['Abbreviation', 'QualifyingTime (s)']].copy()
        
        race_data = pd.merge(results_r[['Abbreviation', 'TeamName', 'FinishingPosition', 'GridPosition']], 
                             qualifying_times, on='Abbreviation', how='left')
        
        race_data['Year'] = year
        race_data['RoundNumber'] = session_r.event['RoundNumber']
        race_data['EventName'] = session_r.event['EventName']
        
        # Calculate Points Index *before* this race using the updated function
        driver_standings, max_driver_points, constructor_standings, max_constructor_points = get_standings_before_round(year, race_data['RoundNumber'].iloc[0])
        
        # Calculate Driver Points Index
        if max_driver_points > 0:
            race_data['PointsIndex'] = race_data['Abbreviation'].apply(lambda x: driver_standings.get(x, 0) / max_driver_points)
        else:
            race_data['PointsIndex'] = 0.0
            
        # Calculate Constructor Points Index
        if max_constructor_points > 0:
            race_data['ConstructorPointsIndex'] = race_data['TeamName'].apply(lambda x: constructor_standings.get(x, 0) / max_constructor_points)
        else:
             race_data['ConstructorPointsIndex'] = 0.0
             
        race_data.rename(columns={'Abbreviation': 'Driver', 'TeamName': 'Team'}, inplace=True)
        
        mean_quali_time = race_data['QualifyingTime (s)'].mean()
        race_data['QualifyingTime (s)'].fillna(mean_quali_time, inplace=True)
        # Ensure no NaNs remain in core features, including the new one
        race_data.dropna(subset=['Driver', 'Team', 'FinishingPosition', 'QualifyingTime (s)', 'PointsIndex', 'ConstructorPointsIndex', 'GridPosition'], inplace=True)
        
        # Include GridPosition in the returned columns
        return race_data[['Year', 'RoundNumber', 'EventName', 'Driver', 'Team', 'QualifyingTime (s)', 'PointsIndex', 'ConstructorPointsIndex', 'GridPosition', 'FinishingPosition']]
        
    except Exception as e:
        print(f"Error processing {year} {event_identifier}: {e}")
        return None

# --- Configuration ---
TARGET_YEAR = 2025
TARGET_GP_NAME = 'Bahrain' # Example: 'Monaco', 'British', 'Saudi Arabian Grand Prix'
HISTORICAL_YEARS = range(TARGET_YEAR - 1, 2018, -1) # e.g., 2024 down to 2019

# --- Data Gathering ---
all_race_data = []

# 1. Get Target GP Info
try:
    schedule_target_year = fastf1.get_event_schedule(TARGET_YEAR, include_testing=False)
    target_event = schedule_target_year[schedule_target_year['EventName'].str.contains(TARGET_GP_NAME, case=False, na=False)]
    
    if target_event.empty:
        raise ValueError(f"Target GP '{TARGET_GP_NAME}' not found in {TARGET_YEAR} schedule.")
        
    target_round = target_event['RoundNumber'].iloc[0]
    target_venue = target_event['Location'].iloc[0]
    target_event_name = target_event['EventName'].iloc[0] # Official name for consistency
    print(f"Target Event: {TARGET_YEAR} {target_event_name} (Round {target_round}, Venue: {target_venue})")

except Exception as e:
    raise ValueError(f"Error finding target GP info: {e}")

# 2. Get Data from Current Season (up to target GP)
print(f"\n--- Fetching {TARGET_YEAR} data before Round {target_round} ---")
schedule_current_year_filtered = schedule_target_year[schedule_target_year['RoundNumber'] < target_round]
for index, race in schedule_current_year_filtered.iterrows():
    print(f"Processing {TARGET_YEAR} {race['EventName']}...")
    data = get_race_data(TARGET_YEAR, race['EventName'])
    if data is not None:
        all_race_data.append(data)

# 3. Get Historical Data
print(f"\n--- Fetching Historical Data ---")
for year in HISTORICAL_YEARS:
    try:
        schedule_hist = fastf1.get_event_schedule(year, include_testing=False)
        print(f"Processing all races for {year}...")
        # Iterate through all races in the historical year's schedule
        for index, race_hist in schedule_hist.iterrows():
            event_name_hist = race_hist['EventName']
            print(f"  Processing {year} {event_name_hist}...")
            data = get_race_data(year, event_name_hist)
            if data is not None:
                all_race_data.append(data)
    except Exception as e:
        print(f"Error getting schedule or processing races for {year}: {e}")

# 4. Combine Data
if not all_race_data:
    raise ValueError("No data could be collected. Check FastF1 setup and GP/Year validity.")

combined_data = pd.concat(all_race_data, ignore_index=True)

# Drop the RoundNumber column
if 'RoundNumber' in combined_data.columns:
    combined_data.drop(columns=['RoundNumber'], inplace=True)
    print("Dropped 'RoundNumber' column.")
else:
    print("'RoundNumber' column not found in combined_data.")

print(f"\n--- Combined Data Shape after dropping RoundNumber: {combined_data.shape} ---")
print(combined_data.head())
print(combined_data.info())
print(combined_data.describe())

# Export the combined dataset to CSV
try:
    csv_filename = 'combined_f1_data.csv'
    combined_data.to_csv(csv_filename, index=False)
    print(f"\n--- Full Combined Dataset exported to {csv_filename} ---")
except Exception as e:
    print(f"\nError exporting data to CSV: {e}")

# 5. Prepare Data for Target Race Prediction
print(f"\n--- Preparing data for {TARGET_YEAR} {target_event_name} prediction ---")
try:
    # Get Qualifying data for the actual target race
    target_q_session = fastf1.get_session(TARGET_YEAR, target_event_name, 'Q')
    target_q_session.load(laps=False, telemetry=False, weather=False, messages=False)
    target_q_results = target_q_session.results
    
    if target_q_results is None or target_q_results.empty:
         raise ValueError(f"Could not load Qualifying results for target event: {TARGET_YEAR} {target_event_name}")
         
    target_q_results['QualifyingTime'] = target_q_results[['Q1', 'Q2', 'Q3']].min(axis=1)
    target_q_results['QualifyingTime (s)'] = target_q_results['QualifyingTime'].dt.total_seconds()
    
    # Get points index before the target race using the updated function
    target_driver_standings, target_max_driver_points, target_constructor_standings, target_max_constructor_points = get_standings_before_round(TARGET_YEAR, target_round)
    
    # Create DataFrame for prediction
    predict_df = target_q_results[['Abbreviation', 'TeamName', 'QualifyingTime (s)']].copy()
    predict_df.rename(columns={'Abbreviation': 'Driver', 'TeamName': 'Team'}, inplace=True)
    
    # Calculate Driver Points Index for prediction set
    if target_max_driver_points > 0:
         predict_df['PointsIndex'] = predict_df['Driver'].apply(lambda x: target_driver_standings.get(x, 0) / target_max_driver_points)
    else:
         predict_df['PointsIndex'] = 0.0
         
    # Calculate Constructor Points Index for prediction set
    if target_max_constructor_points > 0:
         predict_df['ConstructorPointsIndex'] = predict_df['Team'].apply(lambda x: target_constructor_standings.get(x, 0) / target_max_constructor_points)
    else:
         predict_df['ConstructorPointsIndex'] = 0.0
         
    # Handle potential missing Quali times in prediction set (e.g., pit lane start)
    # Use mean from the combined *training* data to fill
    mean_quali_train = combined_data['QualifyingTime (s)'].mean()
    predict_df['QualifyingTime (s)'].fillna(mean_quali_train, inplace=True)
    
    # Sort the qualifying results by time to determine grid positions
    predict_df = predict_df.sort_values('QualifyingTime (s)')
    predict_df['GridPosition'] = range(1, len(predict_df) + 1)
    
    # Ensure no NaNs in prediction features
    predict_df.dropna(subset=['QualifyingTime (s)', 'PointsIndex', 'ConstructorPointsIndex'], inplace=True)
    
    # Keep track of drivers for final output
    predict_drivers = predict_df['Driver'].tolist()
    
    print(f"Prediction input data shape: {predict_df.shape}")
    print(predict_df)
    
except Exception as e:
    raise ValueError(f"Error preparing prediction data: {e}")

core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22', '31', '87', '30', '5', '14', '55', '7', '6']
core           INFO 	Loading data for Australian Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '1', '63', '22', '23', '16', '44', '10', '55', '6', '14', '18', '7', '5', '12', '27', '30', '31', '87']
core           INFO 	Loading data for Chinese Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['81', '4', '63', '1', '31'

Target Event: 2025 Bahrain Grand Prix (Round 4, Venue: Sakhir)

--- Fetching 2025 data before Round 4 ---
Processing 2025 Australian Grand Prix...
Processing 2025 Chinese Grand Prix...


core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22', '31', '87', '30', '5', '14', '55', '7', '6']
core           INFO 	Loading data for Japanese Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '81', '16', '63', '12', '44', '6', '23', '87', '14', '22', '10', '55', '7', '27', '30', '31', '5', '18']
core           INFO 	Loading data for Japanese Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '81', '16', '63',

Processing 2025 Japanese Grand Prix...

--- Fetching Historical Data ---
Processing all races for 2024...
  Processing 2024 Bahrain Grand Prix...


core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '11', '14', '4', '81', '44', '27', '22', '18', '23', '3', '20', '77', '24', '2', '31', '10']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '81', '14', '63', '38', '4', '44', '27', '23', '20', '31', '2', '22', '3', '77', '24', '18', '10']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Qualifying [v3.5.3]
req    

...

  Processing 2019 Brazilian Grand Prix...


core           INFO 	Loading data for Spanish Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['44', '77', '33', '5', '16', '10', '20', '55', '26', '8', '23', '3', '27', '7', '11', '99', '63', '88', '18', '4']
core           INFO 	Loading data for Monaco Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['44', '5', '77', '33', '10', '55', '26', '23', '3', '8', '4', '11', '27', '20', '63', '18', '7', '88', '99', '16']
core           INFO 	Loading data for Canadian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['44', '5', '16', '77', '33', '3', '27'

  Processing 2019 Abu Dhabi Grand Prix...


core           INFO 	Finished loading data for 20 drivers: ['77', '44', '5', '33', '16', '11', '55', '4', '18', '7', '23', '99', '20', '27', '63', '88', '10', '8', '26', '3']
core           INFO 	Loading data for Spanish Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['44', '77', '33', '5', '16', '10', '20', '55', '26', '8', '23', '3', '27', '7', '11', '99', '63', '88', '18', '4']
core           INFO 	Loading data for Monaco Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['44', '5', '77', '33', '10', '55', '26', '23', '3', '8', '4', '11', '27', '20', '63', '18', '7', '88', '99', '16']
core           INFO 	Loading data for Canadian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached 

Dropped 'RoundNumber' column.

--- Combined Data Shape after dropping RoundNumber: (2616, 9) ---
   Year              EventName Driver             Team  QualifyingTime (s)  \
0  2025  Australian Grand Prix    NOR          McLaren              75.096   
1  2025  Australian Grand Prix    VER  Red Bull Racing              75.481   
2  2025  Australian Grand Prix    RUS         Mercedes              75.546   
3  2025  Australian Grand Prix    ANT         Mercedes              76.525   
4  2025  Australian Grand Prix    ALB         Williams              75.737   

   PointsIndex  ConstructorPointsIndex  GridPosition  FinishingPosition  
0          0.0                     0.0             1                  1  
1          0.0                     0.0             3                  2  
2          0.0                     0.0             4                  3  
3          0.0                     0.0            16                  4  
4          0.0                     0.0             6            

In [8]:
# --- Feature Engineering & Model Training (Simplified) ---

# Define features and target
# Using Grid Position, Qualifying Time, Points Index, Constructor Points Index, AvgFinish_Last3, Driver, and Team
features = ['GridPosition', 'QualifyingTime (s)', 'PointsIndex', 'ConstructorPointsIndex', 'Driver', 'Team']
target = 'FinishingPosition'

# Separate features (X) and target (y) from the combined historical/current data
X = combined_data[features].copy()
y = combined_data[target].copy()

# --- One-Hot Encode Categorical Features ---
# This converts 'Driver' and 'Team' names into numerical features the model can use.
# handle_unknown='ignore' ensures the model doesn't crash if a new driver/team appears in the prediction data.
categorical_features = ['Driver', 'Team']
numerical_features = ['GridPosition', 'QualifyingTime (s)', 'PointsIndex', 'ConstructorPointsIndex']

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder on the training data and transform it
X_encoded_cats = encoder.fit_transform(X[categorical_features])
X_encoded_df = pd.DataFrame(X_encoded_cats, columns=encoder.get_feature_names_out(categorical_features), index=X.index)

# Combine numerical features and encoded categorical features for the final training set
X_processed = pd.concat([X[numerical_features], X_encoded_df], axis=1)

print(f"\n--- Processed Training Data Shape: {X_processed.shape} ---")
# print(X_processed.head()) # Uncomment to view processed data

# --- Prepare Prediction Data (Apply the same encoding) ---
# We need to apply the *exact same* transformations to the data for the race we want to predict.
predict_X_numerical = predict_df[numerical_features].copy()
predict_X_categorical = predict_df[categorical_features].copy()

# Use the *fitted* encoder to transform the prediction categorical data
predict_X_encoded_cats = encoder.transform(predict_X_categorical)
predict_X_encoded_df = pd.DataFrame(predict_X_encoded_cats, columns=encoder.get_feature_names_out(categorical_features), index=predict_df.index)

# Combine numerical and encoded categorical features for the prediction set
predict_X_processed = pd.concat([predict_X_numerical, predict_X_encoded_df], axis=1)

# --- Align Columns ---
# Ensure the prediction data has the exact same columns as the training data
# Missing columns (e.g., a driver only in training) are added and filled with 0.
# Extra columns (shouldn't happen with handle_unknown='ignore') are removed.
X_processed_cols = X_processed.columns
predict_X_processed = predict_X_processed.reindex(columns=X_processed_cols, fill_value=0)

print(f"\n--- Processed Prediction Data Shape: {predict_X_processed.shape} ---")
# print(predict_X_processed.head()) # Uncomment to view processed prediction data

# --- Train Gradient Boosting Model ---
# Split the historical/current season data for evaluation (optional but good practice)
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Using Gradient Boosting - a reasonably powerful standard model
# Hyperparameters can be tuned for better performance, but these are sensible defaults.
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
print("\n--- Training Model ---")
model.fit(X_train, y_train)

# --- Predict Finishing Positions for Target Race ---
print(f"\n--- Predicting for {TARGET_YEAR} {target_event_name} ---")
predicted_scores = model.predict(predict_X_processed)

# --- Create Ranked Results ---
# Create a DataFrame with drivers and their raw predicted scores (lower score is better)
results_df = pd.DataFrame({
    'Driver': predict_drivers, 
    'PredictedScore': predicted_scores
})

# Sort drivers by the predicted score (ascending - lower score means better predicted position)
results_df = results_df.sort_values(by='PredictedScore')

# Assign ranks 1, 2, 3... based on the sorted order
results_df['PredictedFinishingPosition'] = np.arange(1, len(results_df) + 1)

# Print final ranked predictions
print(f"\n🏁 Predicted Finishing Order for {TARGET_YEAR} {target_event_name} 🏁\n")
print(results_df[['Driver', 'PredictedFinishingPosition']].to_string(index=False))

# --- Evaluate Model on Test Set ---
print("\n--- Evaluating Model Performance ---")
y_pred_test = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_test)
print(f"🔍 Model Evaluation (Mean Absolute Error on test set): {mae:.2f} positions")
print("(Lower MAE is better. This indicates the average error in predicted position on unseen historical data)")

# --- Feature Importance (Optional) ---
print("\n📊 Top 15 Feature Importances:")
try:
    feature_names = X_processed.columns
    importances = model.feature_importances_
    importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    print(importance_df.head(15).to_string(index=False))
except Exception as e:
    print(f"Could not calculate or display feature importances: {e}")


--- Processed Training Data Shape: (2616, 60) ---

--- Processed Prediction Data Shape: (20, 60) ---

--- Training Model ---

--- Predicting for 2025 Bahrain Grand Prix ---

🏁 Predicted Finishing Order for 2025 Bahrain Grand Prix 🏁

Driver  PredictedFinishingPosition
   PIA                           1
   RUS                           2
   NOR                           3
   ANT                           4
   LEC                           5
   VER                           6
   GAS                           7
   HAM                           8
   TSU                           9
   OCO                          10
   SAI                          11
   DOO                          12
   ALB                          13
   BEA                          14
   ALO                          15
   STR                          16
   HUL                          17
   HAD                          18
   BOR                          19
   LAW                          20

--- Evaluating Model Performan